# Parsing Silk Road Forums HTML files

In [2]:
from bs4 import BeautifulSoup
import datetime
import numpy as np
import pandas as pd
import re
import requests
import codecs
import os
from dateutil.parser import parse

In [22]:
wdir = "/home/abyun/Downloads/w251_proj/silkroad1-forums/practice/"

#Ensure the current directory is correctly set
os.chdir(wdir)
#Build a Pandas data frame to hold the new data
col_names_topic = ["Topic","Date","Author","Author_numposts","Author_type","Author_karma","Response_date",
                   "Responder","Responder_numposts","Responder_type","Responder_karma"]
col_names_profile = ["User","User_numposts","User_type","User_karma","Date_registered","Last_active"]
df_topic = pd.DataFrame(columns = col_names_topic)
df_profile = pd.DataFrame(columns = col_names_profile)

In [23]:
for i in os.listdir(os.getcwd()): #Loops through all files in the current directory
    try:
        if i.find("profile") != -1: #forum profile page
            soup = BeautifulSoup(open(i,'r').read()) 
            
            user_info = soup.find("div",attrs={"class":"username"}) 
            user_block = user_info.find("h4").get_text()
            user = user_block.split(" ")[0]
            user_type = user_info.find("span",attrs={"class":"position"}).get_text()
            
            more_user_info = soup.find("div",attrs={"class":"windowbg2"})
            more_user_info_table = more_user_info.find("div",attrs={"class":"content"}) 
            table_rows = more_user_info_table.find_all("dd")
            user_numposts = table_rows[0].get_text()
            user_karma = table_rows[1].get_text()
            date_registered = parse(table_rows[3].get_text())
            last_active = parse(table_rows[5].get_text())           

            #Creates a numpy array with the scraped values
            new_p = np.array([user,user_numposts,user_type,user_karma,date_registered,last_active])
            #Appends array to data frame
            df_profile = df_profile.append(pd.DataFrame(new_p, index=col_names_profile).transpose())            

        elif i.find("topic") != -1: #forum topic page
            soup = BeautifulSoup(open(i,'r').read()) 

            forum_board = soup.find("div",attrs={"id":"forumposts"})

            #individual posts
            forum_post = forum_board.find_all("div",attrs={"class":["windowbg","windowbg2"]})

            #first post
            main_post = forum_post[0]
            post_info = main_post.find("div",attrs={"class":"postarea"})
            topic_tag = str(post_info.find("a"))
            topic = topic_tag[topic_tag.index(">")+1:topic_tag[1:len(topic_tag)].index("<")+1]
            date_text_tag = str(post_info.find("div",attrs={"class":"smalltext"}))
            date = date_text_tag[date_text_tag.find("/strong>")+9:date_text_tag.find("</div")-2]
            try: 
                date = parse(date)
            except:
                date = parse("0:00")
            poster_info = main_post.find("div",attrs={"class":"poster"})
            author_tag = str(poster_info.find("a"))
            author_text = author_tag.split(" ")[-1]
            author = author_text[author_text.index(">")+1:author_text.index("<")]
            author_numposts_tag = str(poster_info.find("li",{"class":"postcount"}))
            author_numposts_text = author_numposts_tag.split(" ")[-1]
            author_numposts = author_numposts_text[0:author_numposts_text.index("<")] 
            author_type_tag = str(poster_info.find("li",{"class":"postgroup"}))
            author_type_text = author_type_tag.split(" ")[-1]
            author_type = author_type_text[author_type_text.index(">")+1:author_type_text.index("<")] 
            author_karma_tag = str(poster_info.find("li",{"class":"karma"}))
            author_karma_text = author_karma_tag.split(" ")[-1]
            author_karma = author_karma_text[0:author_karma_text.index("<")]

            #response post(s)
            if len(forum_post) > 1:
                for j in range(1, len(forum_post)):
                    response = forum_post[j]
                    responder_info = response.find("div",attrs={"class":"poster"})
                    response_date_text_tag = str(responder_info.find("div",attrs={"class":"smalltext"}))
                    response_date = response_date_text_tag[response_date_text_tag.find("/strong>")+9:response_date_text_tag.find("</div")-2]
                    try: 
                        response_date = parse(response_date)
                    except:
                        response_date = parse("0:00")
                    responder_tag = str(responder_info.find("a"))
                    responder_text = responder_tag.split(" ")[-1]
                    responder = responder_text[responder_text.index(">")+1:responder_text.index("<")]                
                    responder_numposts_tag = str(responder_info.find("li",{"class":"postcount"}))
                    responder_numposts_text = responder_numposts_tag.split(" ")[-1]
                    responder_numposts = responder_numposts_text[0:responder_numposts_text.index("<")] 
                    
                    responder_type_tag = str(responder_info.find("li",{"class":"postgroup"}))
                    responder_type_text = responder_type_tag.split(" ")[-1]
                    responder_type = responder_type_text[responder_type_text.index(">")+1:responder_type_text.index("<")] 
                    responder_karma_tag = str(responder_info.find("li",{"class":"karma"}))
                    responder_karma_text = responder_karma_tag.split(" ")[-1]
                    responder_karma = responder_karma_text[0:responder_karma_text.index("<")]                

                    #Creates a numpy array with the scraped values
                    new_t = np.array([topic, date, author, author_numposts, author_type, author_karma, response_date,
                                   responder, responder_numposts, responder_type, responder_karma])
                    #Appends array to data frame
                    df_topic = df_topic.append(pd.DataFrame(new_t, index=col_names_topic).transpose())

            else:
                response_date = "0:00"
                responder = ""
                responder_numposts = ""
                responder_type = ""
                responder_karma = ""

                #Creates a numpy array with the scraped values
                new_t = np.array([topic, date, author, author_numposts, author_type, author_karma, response_date,
                               responder, responder_numposts, responder_type, responder_karma])
                #Appends array to data frame
                df_topic = df_topic.append(pd.DataFrame(new_t, index=col_names_topic).transpose())
                

        else:
            continue

    except:
        print "File " + i + " has failed"

os.chdir(wdir) #Reset current working directory



File index.php?topic=22162.0 has failed


In [25]:
print df_profile

                 User       User_numposts User_type User_karma  \
0  Rogersslipperyhand   4 (0.017 per day)    Newbie      +0/-0   
0      Psychadelic777   1 (0.004 per day)    Newbie      +0/-0   
0     RiverCityRansom  19 (0.082 per day)    Newbie      +0/-0   

       Date_registered          Last_active  
0  2013-03-15 05:37:00  2013-07-21 01:28:00  
0  2013-03-15 05:39:00  2013-04-04 02:48:00  
0  2013-03-15 05:38:00  2013-04-02 23:41:00  


In [26]:
print df_topic

                                               Topic                 Date  \
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0     Re: Hardly no Cocaine in The Cocaine on SR yet  2012-05-17 18:11:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0     Re: Hardly no Cocaine in The Cocaine on SR yet  2012-05-17 18:11:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
0  ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   

In [27]:

#Remove duplicates from "Feedback" pages
df_profile = df_profile.drop_duplicates(keep="first")
#Re-indexes the dataframe
df_profile.index = range(df_profile.shape[0])

#Remove duplicates from "Feedback" pages
df_topic = df_topic.drop_duplicates(keep="first")
#Re-indexes the dataframe
df_topic.index = range(df_topic.shape[0])

In [33]:
df_profile.to_csv("data_output/df_profile.csv")
df_topic.to_csv("data_output/df_topic.csv")

In [30]:
print df_topic

                                                Topic                 Date  \
0   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
1   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
2   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
3      Re: Hardly no Cocaine in The Cocaine on SR yet  2012-05-17 18:11:00   
4   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
5   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
6   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
7   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
8   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
9   ▌฿ ▌bitcoin fundation memeber www.bitcoratio.c...  2016-03-07 00:00:00   
10                                        need advice  2012-05-07 15:35:00   
11                                        need advice  2012-05-0

# Final script to deploy

In [ ]:
#!/usr/bin/env python

from bs4 import BeautifulSoup
import datetime
import numpy as np
import pandas as pd
import re
import requests
import codecs
import os
from dateutil.parser import parse

#wdir = "/home/abyun/Downloads/w251_proj/silkroad1-forums/practice/"
wdir = "/sandisk1/darknetmarket/silkroad1-forums/2013-11-03/"

#Ensure the current directory is correctly set
os.chdir(wdir)
#Build a Pandas data frame to hold the new data
col_names_topic = ["Topic","Date","Author","Author_numposts","Author_type","Author_karma","Response_date",
                   "Responder","Responder_numposts","Responder_type","Responder_karma"]
col_names_profile = ["User","User_numposts","User_type","User_karma","Date_registered","Last_active"]
df_topic = pd.DataFrame(columns = col_names_topic)
df_profile = pd.DataFrame(columns = col_names_profile)

for i in sorted(os.listdir(os.getcwd())): #Loops through all files in the current directory
    try:
        if i.find("profile") != -1: #forum profile page
            soup = BeautifulSoup(open(i,'r').read()) 
            
            user_info = soup.find("div",attrs={"class":"username"}) 
            user_block = user_info.find("h4").get_text()
            user = user_block.split(" ")[0]
            user_type = user_info.find("span",attrs={"class":"position"}).get_text()
            
            more_user_info = soup.find("div",attrs={"class":"windowbg2"})
            more_user_info_table = more_user_info.find("div",attrs={"class":"content"}) 
            table_rows = more_user_info_table.find_all("dd")
            user_numposts = table_rows[0].get_text()
            user_karma = table_rows[1].get_text()
            date_registered = parse(table_rows[3].get_text())
            last_active = parse(table_rows[5].get_text())           

            #Creates a numpy array with the scraped values
            new_p = np.array([user,user_numposts,user_type,user_karma,date_registered,last_active])
            #Appends array to data frame
            df_profile = df_profile.append(pd.DataFrame(new_p, index=col_names_profile).transpose())            

        elif i.find("topic") != -1: #forum topic page
            soup = BeautifulSoup(open(i,'r').read()) 

            forum_board = soup.find("div",attrs={"id":"forumposts"})

            #individual posts
            forum_post = forum_board.find_all("div",attrs={"class":["windowbg","windowbg2"]})

            #first post
            main_post = forum_post[0]
            post_info = main_post.find("div",attrs={"class":"postarea"})
            topic_tag = str(post_info.find("a"))
            topic = topic_tag[topic_tag.index(">")+1:topic_tag[1:len(topic_tag)].index("<")+1]
            date_text_tag = str(post_info.find("div",attrs={"class":"smalltext"}))
            date = date_text_tag[date_text_tag.find("/strong>")+9:date_text_tag.find("</div")-2]
            try: 
                date = parse(date)
            except:
                date = parse("0:00")
            poster_info = main_post.find("div",attrs={"class":"poster"})
            author_tag = str(poster_info.find("a"))
            author_text = author_tag.split(" ")[-1]
            author = author_text[author_text.index(">")+1:author_text.index("<")]
            author_numposts_tag = str(poster_info.find("li",{"class":"postcount"}))
            author_numposts_text = author_numposts_tag.split(" ")[-1]
            author_numposts = author_numposts_text[0:author_numposts_text.index("<")] 
            author_type_tag = str(poster_info.find("li",{"class":"postgroup"}))
            author_type_text = author_type_tag.split(" ")[-1]
            author_type = author_type_text[author_type_text.index(">")+1:author_type_text.index("<")] 
            author_karma_tag = str(poster_info.find("li",{"class":"karma"}))
            author_karma_text = author_karma_tag.split(" ")[-1]
            author_karma = author_karma_text[0:author_karma_text.index("<")]

            #response post(s)
            if len(forum_post) > 1:
                for j in range(1, len(forum_post)):
                    response = forum_post[j]
                    responder_info = response.find("div",attrs={"class":"poster"})
                    response_date_text_tag = str(responder_info.find("div",attrs={"class":"smalltext"}))
                    response_date = response_date_text_tag[response_date_text_tag.find("/strong>")+9:response_date_text_tag.find("</div")-2]
                    try: 
                        response_date = parse(response_date)
                    except:
                        response_date = parse("0:00")
                    responder_tag = str(responder_info.find("a"))
                    responder_text = responder_tag.split(" ")[-1]
                    responder = responder_text[responder_text.index(">")+1:responder_text.index("<")]                
                    responder_numposts_tag = str(responder_info.find("li",{"class":"postcount"}))
                    responder_numposts_text = responder_numposts_tag.split(" ")[-1]
                    responder_numposts = responder_numposts_text[0:responder_numposts_text.index("<")] 
                    
                    responder_type_tag = str(responder_info.find("li",{"class":"postgroup"}))
                    responder_type_text = responder_type_tag.split(" ")[-1]
                    responder_type = responder_type_text[responder_type_text.index(">")+1:responder_type_text.index("<")] 
                    responder_karma_tag = str(responder_info.find("li",{"class":"karma"}))
                    responder_karma_text = responder_karma_tag.split(" ")[-1]
                    responder_karma = responder_karma_text[0:responder_karma_text.index("<")]                

                    #Creates a numpy array with the scraped values
                    new_t = np.array([topic, date, author, author_numposts, author_type, author_karma, response_date,
                                   responder, responder_numposts, responder_type, responder_karma])
                    #Appends array to data frame
                    df_topic = df_topic.append(pd.DataFrame(new_t, index=col_names_topic).transpose())

            else:
                response_date = "0:00"
                responder = ""
                responder_numposts = ""
                responder_type = ""
                responder_karma = ""

                #Creates a numpy array with the scraped values
                new_t = np.array([topic, date, author, author_numposts, author_type, author_karma, response_date,
                               responder, responder_numposts, responder_type, responder_karma])
                #Appends array to data frame
                df_topic = df_topic.append(pd.DataFrame(new_t, index=col_names_topic).transpose())

        else:
            continue
            
    except:
        print "File " + i + " has failed"

#Remove duplicates and reindex
df_profile = df_profile.drop_duplicates(keep="first")
df_profile.index = range(df_profile.shape[0])

df_topic = df_topic.drop_duplicates(keep="first")
df_topic.index = range(df_topic.shape[0])


df_profile.to_csv("data_output/df_profile.csv", encoding="utf=8")
df_topic.to_csv("data_output/df_topic.csv",encoding="utf-8")
        
os.chdir(wdir) #Reset current working directory